# Creating Your First Graph and Running It in a Session

![9](images/9-1.png)

In [1]:
import tensorflow as tf

print(tf.__version__)

1.8.0


In [2]:
import tensorflow as tf

x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y +y +2

The most important thing to understand is that this code does not actually perform any computation. It just creates a computation graph.

To evaluate this graph, you need to open a TensorFlow session and use it to initialize the variables and evaluate f.

In [3]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

sess.close() # free up resources

42


Having to repeat `sess.run()` all the time is a bit cumbersome(麻烦的), but fortunately there is a better way:

```
with tf.Session() as sess:
    x.initializer().run()
    y.initializer().run()
    result = f.eval()
```

**Error:'Operation' object is not callable.**

In [4]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)

42


Inside the `with` block, the session is set as the default session.

A TensorFlow program is typically split into two parts:
1. **Construction phase**: builds a computation graph representing the ML model and the computations required to train it.
2. **Execution phase**: runs a lood that evaluates a training setp repeatedly, gradually imporving the model parameters.

# Managing Graphs
Any node you create is automatically to the default graph.

In [5]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In most cases this is fine, but sometimes you may want to manage multiple independent graphs. You can do this by creating a new `Graph` and temporarily making it the default fraph inside a `with` block.

In [6]:
graph = tf.Graph()

with graph.as_default():
    x2 = tf.Variable(2)
    
x2.graph is graph

True

In [7]:
x2.graph is tf.get_default_graph()

False

In jupyter, it is common to run the same commands more than once. You may end up with a default graph containing many duplicate nodes. One solution is to restart the kernel while another solution is to just reset the default graph by running `tf.reset_default_graph()`.

# Lifecycle of a Node Value
TensorFlow automatically determines the set of nodes that is depends on and it evaluates these nodes first. For example: 
- First, this code defines a very simple graph. 
- Then it starts a session and runs  the graph to evaluate y: TensorFlow automatically detects that y depends on w, which depends on x, so it first evaluates w, then x, then y, and return the value of y.
- Finally, the code runs the graph to evaluate z.

**It is important to note that it will not reuse the result of the previous evaluation of w and x. In short, the code evaluates w and x twice.**

In [8]:
w = tf.constant(3)
x = w+2
y = x+5
z = x*3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


**All node values are dropped between graph runs, except variable values, which are maintained by the session across graph runs. A variable starts its life when its initializer is run, and it ends when the session is closed.**

to Evaluate y and z efficiently, without evaluating w and x twice as in the previous code, ask TensorFlow to evaluate boty y and z in just one graph run.

In [9]:
with tf.Session() as sess:
    y_val,z_val = sess.run([y,z])
    print(y_val)
    print(z_val)

10
15


In single-process TensorFlow, each session would have its own copy of every variable. In distributed TensorFlow, variable state is stored on the servers, not in the sessions, so multiple sessions can share the same varibles.

# Linear Regression with TensorFlow
TensorFlow operations(*ops* for short) can take any number of inputs and produce any number of outputs. **Constants and variables** take no input(they are called *source ops*). The inputs and outputs are multidimensional arrays, called *tensors*. In the python API tensors are simply represented by Numpy ndarrays. They typically contain floats, but you can also use them to carry strings.

In [13]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m,n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    print(theta_value)

[[ -3.71851807e+01]
 [  4.36337471e-01]
 [  9.39523336e-03]
 [ -1.07113101e-01]
 [  6.44792199e-01]
 [ -4.03380000e-06]
 [ -3.78137082e-03]
 [ -4.23484027e-01]
 [ -4.37219113e-01]]


# Implementing Gradient Descent
**When using Gradient Descent, it is important to first bormalize the input feature vectors, or else trining may be much slower.**


## Manually Computing the Gradient
- `random_uniform()` function creates a node in the graph that will generate a tensor containing random values.
- `assign()` function creates a node that will assign a new value to a variable.
- The main loop executes the training step over and over again(`n_epochs` times) and every 100 iterations it prints out the current Mean Squared Error(mse).

In [18]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
scaled_housing_data_plus_bias = min_max_scaler.fit_transform(housing_data_plus_bias)

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0,1.0), name='theta')
y_pred = tf.matmul(X, theta, name= 'predictions')
error = y_pred -y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta-learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 ==0:
            print('Epoch', epoch, 'MSE=', mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE= 7.2741
Epoch 100 MSE= 1.69565
Epoch 200 MSE= 1.32067
Epoch 300 MSE= 1.24903
Epoch 400 MSE= 1.19984
Epoch 500 MSE= 1.15639
Epoch 600 MSE= 1.11716
Epoch 700 MSE= 1.0816
Epoch 800 MSE= 1.04928
Epoch 900 MSE= 1.01982


## Using autodiff
**TensorFlow's autodiff feature can automatically and efficiently compute the gradients, simply replace the `gradients = ...` line with `gradients=tf.gradients(mse, [theta])[0]`** 

The `gradients()` function takes an op(in this case mse) and a list of variables(in this case just theta), and it creates a list of ops(one per variable) to compute the gradients of the op with regards to each variable.

![9](images/t9-2.png)

In [20]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
scaled_housing_data_plus_bias = min_max_scaler.fit_transform(housing_data_plus_bias)

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0,1.0), name='theta')
y_pred = tf.matmul(X, theta, name= 'predictions')
error = y_pred -y
mse = tf.reduce_mean(tf.square(error), name='mse')
#----------------------------
gradients = tf.gradients(mse, [theta])[0]
#----------------------------
training_op = tf.assign(theta, theta-learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 ==0:
            print('Epoch', epoch, 'MSE=', mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE= 9.36744
Epoch 100 MSE= 1.91181
Epoch 200 MSE= 1.42061
Epoch 300 MSE= 1.33494
Epoch 400 MSE= 1.27868
Epoch 500 MSE= 1.22938
Epoch 600 MSE= 1.185
Epoch 700 MSE= 1.14483
Epoch 800 MSE= 1.1083
Epoch 900 MSE= 1.07499


## Using an Optimizer
TensorFlow computes the gradients for us, and it could be easier. Replace the preceding `gradients = ..` and `training_op = ...` lines with the following code:

```
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)
```

Momentum optimizer often converges much faster than GD.
```
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
```

In [21]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
scaled_housing_data_plus_bias = min_max_scaler.fit_transform(housing_data_plus_bias)

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0,1.0), name='theta')
y_pred = tf.matmul(X, theta, name= 'predictions')
error = y_pred -y
mse = tf.reduce_mean(tf.square(error), name='mse')
#----------------------------
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)
#----------------------------

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 ==0:
            print('Epoch', epoch, 'MSE=', mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE= 7.07728
Epoch 100 MSE= 1.79089
Epoch 200 MSE= 1.42029
Epoch 300 MSE= 1.33912
Epoch 400 MSE= 1.28144
Epoch 500 MSE= 1.23088
Epoch 600 MSE= 1.18562
Epoch 700 MSE= 1.14483
Epoch 800 MSE= 1.1079
Epoch 900 MSE= 1.07432


# Feeding Data to the Training Algorithm
To implement Mini-batch Gradient Descent, we need find a way to replace X and y at every iteration with the next mini-batch. The simplest way to do this is to use placeholder nodes. These nodes are special because they don't actually perform any computation, they just output the data you tell them to output at runtime. They are typically used to pass the training data to TensorFlow during training.

To create a placeholder, you must call the `placeholder()` function and specify the output tensor's data type. Shape is optional.

In [23]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A+5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A:[[1,2,3]]})
    B_val_2 = B.eval(feed_dict={A:[[4,5,6],[7,8,9]]})
    
print(B_val_1)
print(B_val_2)

[[ 6.  7.  8.]]
[[  9.  10.  11.]
 [ 12.  13.  14.]]


![9](images/c9-1.png)

# Saving and Restoring Models
- Save trained models.
- Save checkpoints at regular intervals during training so that if your computer crashes during training you can continue from the last checkpoint rather than start over from scratch.

#### Two steps to save models:
1. create a `Saver` node at the end of the construction phase(after all variable nodes are created)
2. In the execution phase, call its `save()` method, passint ti the session and path of the checkpoint file.

![9](images/c9-2.png)

#### restoring a model
1. create a `Saver` node at the end of the construction phase(after all variable nodes are created)
2. At the begining of execution phase, call `restore()` method of `Saver()` object instead of initializing the variables.

```
with tf.Session() as sess:
    saver.restore(sess, '/tmp/mymodel_final.ckpt')
    [...]
```

By default a `Saver` saves and restores all variables under their own name, but we can specify which variables to save or restore, and what name to use.
```
saver = tf.train.Saver({'Weights':theta})
```

# Visualizing the Graph and Training Curves Using TensorBoard
- Step 1: write the graph definition and some training stats to a log directory that TensorBoard will read from. Use a different log directory every time you run your program, or else TensorBoard will merge stats from different runs, which will mess up the visualizations. The simplest solution for this is to include a timestamp in the log dirctory name.

In [25]:
from datetime import datetime

now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = 'tf_logs'
log_dir = '{}/run-{}/'.format(root_logdir, now)

- Step 2: add the following code at the very end of the construction phase:
    - first line: creates a node in the graph that will evaluate the MSE value and write it to a TensorBoard-compatible binary log string called a *summary*
    - 2nd line: creates a `FileWriter` that you will use to write summaries to logfiles in the log girectory.

In [26]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

- Step 3: update the execution phase to evaluate the `mse_summary` node regularly during training. This will output a summary that you can then write the events file using the `file_writer`.

![9](images/c9-3.png)

**Avoid logging training stats at every single training step, as this would significantly slow down training.**

- Step 4: close the `FileWriter` at the end of the program.
```
file_writer.close()
```

Use `tensorboard --logdir tf_logs/` to TensorBoard!

# Name Scopes
Create `name scopes` to group related nodes.

```
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name='mse')
```

# Modularity
